## PLX - Phuture Liquid Staking Index - Methodology Template

## Configuring packages

In [1]:
import pandas as pd
import numpy as np
import requests
import decouple
import sys 
sys.path.append('../')
import datetime 
import time
from methodology import Methodology

In [2]:
from pycoingecko import CoinGeckoAPI
key =  decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key=key)

In [3]:
import plotly.express as px
import plotly.graph_objects as go

In [4]:
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

## Index Variables

In [5]:
min_mcap =  0.5e8
min_weight = 0.005
max_weight = 0.3
circ_supply_threshold = 0.2
liveness_threshold = 180 # In days
liquidity_consistency = 90 # In days
coingecko_category = "liquid-staking-governance-tokens"

# Calculating max slippage based on liquidity threshold
liq = 1.5*1e6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e2
trade_value = 1e5
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .01
max_slippage = round(max_slippage, 2)
max_slippage

stablecoins = pd.DataFrame(cg.get_coins_markets('usd',category='stablecoins')).set_index('id')
ctokens = pd.DataFrame(cg.get_coins_markets('usd',category='compound-tokens')).set_index('id')
atokens = pd.DataFrame(cg.get_coins_markets('usd',category='aave-tokens')).set_index('id')

methodology_class = Methodology(min_mcap,min_weight,max_weight,circ_supply_threshold,liveness_threshold,liquidity_consistency,max_slippage,[stablecoins,ctokens,atokens],coingecko_category)

## Data retrieval and filtering

### Token inclusion criteria

1. **Project and token characteristics**
    1. The project’s token should have been listed on CoinGecko with pricing data spanning at least 6 months prior to the date of inclusion in the index.
    1. The project should have an active community.
    1. No rebasing or deflationary tokens.
    1. Synthetic tokens which derive their value from external price feeds are not permissible. 
    1. The project’s token must not have the ability to pause token transfers.
    1. The project must have a minimum circulating market cap of $150mm.
   <br>
1. **Pricing requirmeents**
    1. Token should have a reliable price feed from one of our authorised providers. 
    <br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 30% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The price should experience no more than 4% price impact when executing a $100k trade.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.
1. **Weighting requirements**
    1. The maximum weight any one token can have is 30%.
    1. All excess weight is proportionally redistributed to all uncapped tokens. After this has been completed, if another token now exceeds the 30% threshold the excess will be redistributed to the remaining uncapped tokens. This process will occur iteratively until there are no tokens that exceed the maximum weight.
    1. Any asset with a weight below 0.5% will be removed from the index.


#### Supported assets and blockchains

#### CoinGecko

##### Gathering market data

In [6]:
methodology_class.get_category_data()

,symbol,name,current_price,market_cap,market_cap_rank,fully_diluted_valuation,circulating_supply,total_supply,max_supply
id,,,,,,,,,
lido-dao,ldo,Lido DAO,2.400000,2.133064e+09,33.0,2.396570e+09,8.900483e+08,1.000000e+09,1.000000e+09
frax-share,fxs,Frax Share,7.790000,5.867055e+08,90.0,7.761486e+08,7.535113e+07,9.968150e+07,9.968150e+07
rocket-pool,rpl,Rocket Pool,27.310000,5.435445e+08,99.0,5.435445e+08,1.991675e+07,1.991675e+07,NaN
ankr,ankr,Ankr Network,0.025658,2.568294e+08,173.0,2.568294e+08,1.000000e+10,1.000000e+10,1.000000e+10
stride,strd,Stride,1.510000,1.332730e+08,270.0,1.332730e+08,8.826361e+07,8.826361e+07,1.000000e+08
bifrost,bfc,Bifrost,0.052261,7.239713e+07,412.0,1.238404e+08,1.384675e+09,2.368584e+09,2.368584e+09
veno-finance,vno,Veno Finance,0.161199,6.626727e+07,437.0,2.966078e+08,4.098923e+08,1.834650e+09,2.000000e+09
marinade,mnde,Marinade,0.255244,6.142066e+07,457.0,2.550896e+08,2.407807e+08,1.000000e+09,1.000000e+09


In [7]:
methodology_class.add_asset_to_category(['benqi','stader','stakewise','stake-dao','yearn-finance','yield-yak'])

,symbol,name,current_price,market_cap,market_cap_rank,fully_diluted_valuation,circulating_supply,total_supply,max_supply
id,,,,,,,,,
lido-dao,ldo,Lido DAO,2.400000,2.133064e+09,33.0,2.396570e+09,8.900483e+08,1.000000e+09,1.000000e+09
frax-share,fxs,Frax Share,7.790000,5.867055e+08,90.0,7.761486e+08,7.535113e+07,9.968150e+07,9.968150e+07
rocket-pool,rpl,Rocket Pool,27.310000,5.435445e+08,99.0,5.435445e+08,1.991675e+07,1.991675e+07,NaN
yearn-finance,yfi,yearn.finance,8505.850000,2.824510e+08,163.0,3.116659e+08,3.322901e+04,3.666600e+04,3.666600e+04
ankr,ankr,Ankr Network,0.025658,2.568294e+08,173.0,2.568294e+08,1.000000e+10,1.000000e+10,1.000000e+10
stride,strd,Stride,1.510000,1.332730e+08,270.0,1.332730e+08,8.826361e+07,8.826361e+07,1.000000e+08
bifrost,bfc,Bifrost,0.052261,7.239713e+07,412.0,1.238404e+08,1.384675e+09,2.368584e+09,2.368584e+09
veno-finance,vno,Veno Finance,0.161199,6.626727e+07,437.0,2.966078e+08,4.098923e+08,1.834650e+09,2.000000e+09
marinade,mnde,Marinade,0.255244,6.142066e+07,457.0,2.550896e+08,2.407807e+08,1.000000e+09,1.000000e+09


In [8]:
methodology_class.get_all_coin_data()
methodology_class.filter_and_merge_coin_data()
methodology_class.token_supply_check()
methodology_class.asset_maturity_check()
methodology_class.remove_manual_exclusions([])

,symbol,name,current_price,market_cap,market_cap_rank,fully_diluted_valuation,circulating_supply,total_supply,max_supply,platforms
id,,,,,,,,,,
lido-dao,ldo,Lido DAO,2.400000,2.133064e+09,33.0,2.396570e+09,8.900483e+08,1.000000e+09,1.000000e+09,{'ethereum': '0x5a98fcbea516cf06857215779fd812...
frax-share,fxs,Frax Share,7.790000,5.867055e+08,90.0,7.761486e+08,7.535113e+07,9.968150e+07,9.968150e+07,{'ethereum': '0x3432b6a60d23ca0dfca7761b7ab564...
rocket-pool,rpl,Rocket Pool,27.310000,5.435445e+08,99.0,5.435445e+08,1.991675e+07,1.991675e+07,NaN,{'ethereum': '0xd33526068d116ce69f19a9ee46f0bd...
yearn-finance,yfi,yearn.finance,8505.850000,2.824510e+08,163.0,3.116659e+08,3.322901e+04,3.666600e+04,3.666600e+04,{'ethereum': '0x0bc529c00c6401aef6d220be8c6ea1...
ankr,ankr,Ankr Network,0.025658,2.568294e+08,173.0,2.568294e+08,1.000000e+10,1.000000e+10,1.000000e+10,{'ethereum': '0x8290333cef9e6d528dd5618fb97a76...
bifrost,bfc,Bifrost,0.052261,7.239713e+07,412.0,1.238404e+08,1.384675e+09,2.368584e+09,2.368584e+09,{'ethereum': '0x0c7d5ae016f806603cb1782bea29ac...
benqi,qi,BENQI,0.008222,3.460599e+07,625.0,5.913111e+07,4.213741e+09,7.200000e+09,7.200000e+09,{'avalanche': '0x8729438eb15e2c8b576fcc6aecda6...
stakewise,swise,StakeWise,0.093409,3.043610e+07,664.0,9.340854e+07,3.258385e+08,1.000000e+09,1.000000e+09,{'ethereum': '0x48c3399719b582dd63eb5aadf12a40...
stader,sd,Stader,0.626069,2.050539e+07,789.0,9.367202e+07,3.283593e+07,1.500000e+08,1.500000e+08,{'ethereum': '0x30d20208d987713f46dfd34ef128bb...


In [9]:
methodology_class.assess_liquidity()
methodology_class.slippage_data

0x7205705771547cf79201111b4bd8aaf29467b9ec
0x9046d36440290ffde54fe0dd84db8b1cfee9107b
0x9eaf8c1e34f05a589eda6bafdf391cf6ad3cb239
0xdf474b7109b73b7d57926d43598d5934131136b2
0xaeaeed23478c3a4b798e4ed40d8b7f41366ae861
0xaeaeed23478c3a4b798e4ed40d8b7f41366ae861
0x84c882a4d8eb448ce086ea19418ca0f32f106117


,spot price,delivery price,slippage,blockchain
id,,,,
rocket-pool,0.036717,0.036629,-0.002378,ethereum
lido-dao,0.416755,0.414002,-0.006607,ethereum
yearn-finance,0.000119,0.000116,-0.025408,ethereum
stakewise,10.611340,10.258610,-0.033241,ethereum
stader,1.599620,1.541104,-0.036581,ethereum
frax-share,0.132154,0.126821,-0.040353,ethereum
benqi,121.085819,109.859218,-0.092716,avalanche
yield-yak,0.001603,0.001281,-0.200892,avalanche
stake-dao,2.744022,2.093094,-0.237217,ethereum


In [10]:
methodology_class.check_redstone_price_feeds()

Dropping stader because a price feed is unavailable


,symbol,name,current_price,market_cap,market_cap_rank,fully_diluted_valuation,circulating_supply,total_supply,max_supply,platforms
id,,,,,,,,,,
lido-dao,ldo,Lido DAO,2.400000,2.133064e+09,33.0,2.396570e+09,8.900483e+08,1.000000e+09,1.000000e+09,{'ethereum': '0x5a98fcbea516cf06857215779fd812...
frax-share,fxs,Frax Share,7.790000,5.867055e+08,90.0,7.761486e+08,7.535113e+07,9.968150e+07,9.968150e+07,{'ethereum': '0x3432b6a60d23ca0dfca7761b7ab564...
rocket-pool,rpl,Rocket Pool,27.310000,5.435445e+08,99.0,5.435445e+08,1.991675e+07,1.991675e+07,NaN,{'ethereum': '0xd33526068d116ce69f19a9ee46f0bd...
yearn-finance,yfi,yearn.finance,8505.850000,2.824510e+08,163.0,3.116659e+08,3.322901e+04,3.666600e+04,3.666600e+04,{'ethereum': '0x0bc529c00c6401aef6d220be8c6ea1...
benqi,qi,BENQI,0.008222,3.460599e+07,625.0,5.913111e+07,4.213741e+09,7.200000e+09,7.200000e+09,{'avalanche': '0x8729438eb15e2c8b576fcc6aecda6...
stakewise,swise,StakeWise,0.093409,3.043610e+07,664.0,9.340854e+07,3.258385e+08,1.000000e+09,1.000000e+09,{'ethereum': '0x48c3399719b582dd63eb5aadf12a40...


In [11]:
methodology_class.calculate_weights()

            lido-dao  frax-share  rocket-pool  yearn-finance  ankr  bifrost  \
date                                                                          
2023-12-02       0.3         NaN          NaN            NaN   NaN      NaN   

            benqi  stakewise  stader  stake-dao  yield-yak  
date                                                        
2023-12-02    NaN        NaN     NaN        NaN        NaN  


ValueError: Boolean array expected for the condition, not float64